# Librerias

In [435]:
import pandas as pd
import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine, text, insert, Table, Column, Integer, String, MetaData, BIGINT

In [436]:
from warnings import filterwarnings
filterwarnings('ignore')

# Funciones globales

In [437]:
def nombre_estados(fila):
    return fila[:fila.find('!')]

In [438]:
name_columns = {
    'Label (Grouping)': 'name',
    'Arizona!!Estimate': 'Arizona',
    'Florida!!Estimate': 'Florida',
    'Georgia!!Estimate': 'Georgia', 
    'Illinois!!Estimate': 'Illinois', 
    'Michigan!!Estimate': 'Michigan',
    'North Carolina!!Estimate': 'North Carolina',
    'Ohio!!Estimate': 'Ohio',
    'Tennessee!!Estimate': 'Tennessee',
    'Texas!!Estimate': 'Texas',
    'Virginia!!Estimate': 'Virginia'
}

In [439]:
def uniones(tabla, etq):
    columnas = ["Arizona", "Florida", "Georgia", "Illinois", "Michigan", "North Carolina", "Ohio", "Tennessee", "Texas", "Virginia"] #Pensylvania, New York, California
    for col in columnas:
        tabla[f"{col}_x"] += tabla[f"{col}_y"]
    tabla = tabla.iloc[:, :11].assign(nombre=etq)
    return tabla

In [440]:
def separar(tabla, estado, etiq):
    tabla = tabla[["name",estado, "nombre"]]
    tabla = tabla.rename(columns={estado: "total"})
    tabla = tabla.assign(etiqueta=etiq)
    return tabla

In [441]:
def princ(tabla):
    state_mappings = [("Arizona_x", "Arizona"), ("Florida_x", "Florida"), ("Georgia_x", "Georgia"),
                      ("Illinois_x", "Illinois"), ("Michigan_x", "Michigan"), ("North Carolina_x", "North Carolina"),
                      ("Ohio_x", "Ohio"), ("Tennessee_x", "Tennessee"),
                      ("Texas_x", "Texas"), ("Virginia_x", "Virginia"), 
                      ]
    tables = []
    for state, state_code in state_mappings:
        table = separar(tabla, state, state_code)
        tables.append(table)
    tabla__ = pd.concat(tables)
    return tabla__

In [442]:
def limpiar_edad(fila):
    return fila.strip()

In [443]:
def general(etiq, one, two):
    tabla = pd.merge(one, two, on="name")
    tabla = uniones(tabla, etiq)
    tabla = princ(tabla)
    return tabla

In [444]:
def rangos(edades, inicial, porcentaje, etiqueta1, etiqueta2):
    rango_aum = edades[inicial:inicial+1]
    rango_aum["name"] = etiqueta1
    rango_aum2 = rango_aum.iloc[:,-10:] * porcentaje
    rango_aum2 = rango_aum2.astype(int)
    rango_aum.iloc[:,-10:] = rango_aum2
    edades = pd.concat([edades, rango_aum])

    rango_m = rango_aum.iloc[:,-10:] * (1-porcentaje)
    rango_m = rango_m.astype(int)
    rango_aum.iloc[:,-10:] = rango_m
    rango_aum["name"] = etiqueta2
    edades[inicial:inicial+1]= rango_aum
    return edades

In [445]:
def etnia(tabla, etiqu):
    
    tabla = tabla.rename(columns=name_columns)
    tabla.iloc[:,-10:] = tabla.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
    tabla["name"] = tabla["name"].apply(limpiar_edad)
    tabla['name'] = tabla['name'].replace('18 and 19 years', '18 to 19 years')
    male_tabla = tabla[2:16]
    female_tabla = tabla[17:]

    tablas_etnias = [male_tabla, female_tabla]

    args_list = [(8, 0.65, "35 to 39 years", "40 to 44 years"), (9, 0.55, "45 to 49 years", "50 to 54 years"),
                (10, 0.55, "55 to 59 years", "60 to 64 years"), (11, 0.55, "65 to 69 years", "70 to 74 years"),
                (12, 0.55, "75 to 79 years", "80 to 44 years")]

    for t_etnia in range(len(tablas_etnias)):
        for args in args_list:
            tablas_etnias[t_etnia] = rangos(tablas_etnias[t_etnia], *args)

    tabla = general(etiqu, tablas_etnias[0], tablas_etnias[1])
    return tabla

In [446]:
def ampliar(tabla, n, rangos):
    for i in range(0,n):
        tabla = pd.concat([tabla, tabla])
    tabla = tabla[0:n] 
    tabla["name"] = rangos
    return tabla

In [447]:
def ultimos(tabla, porcentaje):
    tabla_2 = tabla["total"]
    tabla_2 = tabla_2 * porcentaje
    tabla_2 = tabla_2.astype(int)
    tabla["total"] = tabla_2
    return tabla

In [448]:
def ids_merge(tabla, estados, edades):
    tabla = tabla.rename(columns={
        "etiqueta":"estados", "name":"edad"
    })
    tabla1 = pd.merge(tabla, estados, how="left" ,on=["estados"])
    tabla1 = tabla1.rename(columns={
        "index":"estado"
    })
    tabla1 = pd.merge(tabla1, edades, how="left" ,on=["edad","estado"])
    return tabla1

In [449]:
def persona(one_r):
    one2= one_r[0:1]
    one2= ampliar(one2, 9, ["15 to 17 years", "18 to 19 years", "20 to 24 years", "25 to 29 years", 
                            "30 to 34 years", "35 to 39 years", "40 to 44 years", "45 to 49 years",
                            "50 to 54 years"])

    one3 = one_r[3:4]
    one3 = ampliar(one3, 3, ["75 to 79 years", "80 to 84 years", "85 years and over"])
    
    one4 = one_r[1:2]
    one4 = ampliar(one4, 2, ["55 to 59 years", "60 to 64 years"])

    one5 = one_r[2:3]
    one5 = ampliar(one5, 2, ["65 to 69 years", "70 to 74 years"])

    one_r = pd.concat([one2, one4])
    one_r = pd.concat([one_r, one5])
    one_r = pd.concat([one_r, one3])

    return one_r

In [450]:
def age_range_to_list(age_range_str):
    age_range = re.findall(r"\d+", age_range_str)
    age_range = [int(x) for x in age_range]
    start = age_range[0]
    if(len(age_range) > 1):
        end = age_range[1]
        age_list = [(i, i+4) for i in range(start, end+1, 5)]
    elif(len(age_range) == 1):
        if(start == 65):
            age_list = [(65,69),(70,74),(75,79),(80,84),(85)]
        elif(start == 25):
            age_list = [(5),(5,9),(10,14),(15,17),(18,19),(20,24)]
    return age_list

In [451]:
def copay_tranform(filas):
    if(filas != filas):
        filas = "$0.00"
    elif(filas == "Not Applicable"):
        filas = "$0.00"
    elif(filas == "No Charge"):
        filas = "$0.00"
    elif(filas == "No Charge after deductible"):
        filas = "$0.00"
    return filas

In [452]:
def codigo_a_estado(fila):
    if(fila == "TX"):
        fila = "Texas"
    elif(fila == "FL"):
        fila = "Florida"
    elif(fila == "NY"):
        fila = "New York"
    elif(fila == "PA"):
        fila = "Pennsylvania"
    elif(fila == "IL"):
        fila = "Illinois"
    elif(fila == "OH"):
        fila = "Ohio"
    elif(fila == "GA"):
        fila = "Georgia"
    elif(fila == "NC"):
        fila = "North Carolina"
    elif(fila == "MI"):
        fila = "Michigan"
    elif(fila == "CA"):
        fila = "California"
    elif(fila == "VA"):
        fila = "Virginia"
    elif(fila == "AZ"):
        fila = "Arizona"
    elif(fila == "TN"):
        fila = "Tennessee"
    return fila

In [453]:
state_filter = ["TX", "FL", "NY", "PA", "IL", "OH", "GA", "NC", "MI", "CA", "VA", "AZ", "TN"]

# Censo

## ESTADOS

In [454]:
poblacion = pd.read_csv("data_censo/poblacion.csv")

In [455]:
estados = pd.DataFrame(poblacion.iloc[1,1:11])
estados.iloc[:,-10:] = estados.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
estados = estados.reset_index()
estado = []
for i in estados["index"].tolist():
     estado.append(i[:i.find('!')])
estados = pd.DataFrame({ "estados": estado,
                         "poblacion":estados[1].astype(int)
})
estados = estados.reset_index()
estados.rename(columns={"index":"id"})
estados

,index,estados,poblacion
0,0,Arizona,7276316
1,1,Florida,21781128
2,2,Georgia,10799566
3,3,Illinois,12671469
4,4,Michigan,10050811
5,5,North Carolina,10551162
6,6,Ohio,11780017
7,7,Tennessee,6975218
8,8,Texas,29527941
9,9,Virginia,8642274


## Edades

In [456]:
poblacion_edades = pd.read_csv("data_censo/edades.csv")

In [457]:
edades = pd.DataFrame(poblacion_edades.iloc[2:20,0:])
edades.iloc[:,-10:] = edades.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))

In [458]:
# Ampliando rango de edades
rango_aum = edades[3:4]
rango_aum["Label (Grouping)"] = "15 to 17 years"
rango_aum2 = rango_aum.iloc[:,-10:] * 0.65
rango_aum2 = rango_aum2.astype(int)
rango_aum.iloc[:,-10:] = rango_aum2
edades = pd.concat([edades, rango_aum])

rango_m = rango_aum.iloc[:,-10:] * 0.35
rango_m = rango_m.astype(int)
rango_aum.iloc[:,-10:] = rango_m
rango_aum["Label (Grouping)"] = "18 to 19 years"
edades[3:4]= rango_aum

In [459]:
edades = pd.melt(edades, id_vars=['Label (Grouping)'], value_vars=edades.columns.to_list()[1:], var_name='estados', value_name='cantidad')
edades["estados"] = edades["estados"].apply(nombre_estados)
edades.columns = ["edad","estados","cantidad"]
edades = pd.merge(edades, estados, how="left" ,on=["estados"])
edades = edades[["index","edad","cantidad"]].reset_index()
edades.columns = ["id","estado","edad","cantidad"]
edades["id"] = edades["id"].astype(str) + edades["estado"].astype(str)
edades["edad"] = edades["edad"].apply(limpiar_edad)
edades.head(3)

,id,estado,edad,cantidad
0,00,0,Under 5 years,402255
1,10,0,5 to 9 years,446309
2,20,0,10 to 14 years,476245


## Etnias

In [460]:
# poblacion_etnias_white = pd.read_csv("data_censo/blancos.csv")
# poblacion_etnias_black = pd.read_csv("data_censo/black.csv")
# poblacion_etnias_american = pd.read_csv("data_censo/etnia_american.csv")
# poblacion_etnias_asian = pd.read_csv("data_censo/etnia_asian.csv")

In [461]:
# etnias_white = etnia(poblacion_etnias_white, "WHITE ALONE")
# etnias_black = etnia(poblacion_etnias_black, "BLACK OR AFRICAN AMERICAN ALONE")
# etnias_american = etnia(poblacion_etnias_american, "AMERICAN INDIAN AND ALASKA NATIVE ALONE")
# etnias_asian = etnia(poblacion_etnias_asian, "ASIAN ALONE")

In [462]:
# tablas_etnias = [etnias_white, etnias_black, etnias_american, etnias_asian]
# tablas_pop_etnias = pd.concat(tablas_etnias)
# tablas_pop_etnias = tablas_pop_etnias.reset_index().reset_index()
# tablas_pop_etnias = tablas_pop_etnias[["level_0", "name", "etiqueta", "nombre", "total"]]
# tablas_pop_etnias['name'] = tablas_pop_etnias['name'].replace('80 to 44 years', '80 to 84 years')

In [463]:
# tablas_pop_etnias = ids_merge(tablas_pop_etnias, estados, edades)

In [464]:
# tablas_pop_etnias = tablas_pop_etnias[["level_0", "id", "nombre", "total"]]
# tablas_pop_etnias.columns = ["id", "edad_id", "etnia", "cantidad_etnias"]
# tablas_pop_etnias.head(3)

## Estado Civil

In [465]:
civil = pd.read_csv("data_censo/estado_civil.csv")

In [466]:
civil = civil.rename(columns=name_columns)
civil.iloc[:,-10:] = civil.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
civil["name"] = civil["name"].apply(limpiar_edad)
civil['name'] = civil['name'].replace('18 and 19 years', '18 to 19 years')

In [467]:
male_never = civil[3:17]
male_married = civil[19:33]
male_separated = civil[35:49]
male_union = civil[50:64]
male_widowed = civil[65:79]
male_divorced = civil[80:94]

female_never = civil[96:110]
female_married = civil[112:126]
female_separated = civil[128:142]
female_union = civil[143:157]
female_widowed = civil[158:172]
female_divorced = civil[173:]

In [468]:
tablas_civil = [male_never, male_married, male_separated, male_union, male_widowed, male_divorced,
                female_never, female_married, female_separated, female_union, female_widowed, female_divorced]

for sex in range(len(tablas_civil)):
    tablas_civil[sex] = rangos(tablas_civil[sex], 11, 0.65, "65 to 69 years", "70 to 74 years")
    tablas_civil[sex] = rangos(tablas_civil[sex], 12, 0.55, "75 to 79 years", "80 to 84 years")

In [469]:
names = ["Soltero", "Casado", "Separado", "Union", "Viudo", "Divorciado"]
data = [(tablas_civil[0], tablas_civil[6]), (tablas_civil[1], tablas_civil[7]), (tablas_civil[2], tablas_civil[8]),
        (tablas_civil[3], tablas_civil[9]), (tablas_civil[4], tablas_civil[10]), (tablas_civil[5], tablas_civil[11])]

estado_civil = pd.DataFrame()

for name, d in zip(names, data):
    df = general(name, d[0], d[1])
    estado_civil = pd.concat([estado_civil, df])

In [470]:
estado_civil = estado_civil.reset_index().reset_index()
estado_civil = estado_civil[["level_0", "name", "etiqueta", "nombre", "total"]]
tabla_estado_civil = ids_merge(estado_civil, estados, edades)
tabla_estado_civil = tabla_estado_civil[["level_0", "id", "nombre", "total"]]
tabla_estado_civil.columns = ["id", "edad_id", "estado_civil", "cantidad_civil"]
tabla_estado_civil.head(3)

,id,edad_id,estado_civil,cantidad_civil
0,0,180,Soltero,286865
1,1,30,Soltero,189569
2,2,40,Soltero,431735


## Tamaño Hogar

In [471]:
hogar = pd.read_csv("data_censo/tam_hogar.csv")

In [472]:
hogar = hogar.rename(columns=name_columns)
hogar.iloc[:,-10:] = hogar.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
one = hogar[3:7]
two = hogar[8:12]

one_r = hogar[14:18]
two_r = hogar[19:]

In [473]:
one = persona(one)
one_r = persona(one_r)
two = persona(two)
two_r = persona(one_r)

In [474]:
one = general("1 persona", one, one_r)
two = general("2 o mas personas", two, two_r)
tam_hogar = pd.concat([one, two])
tam_hogar = tam_hogar.reset_index().reset_index()
tam_hogar = tam_hogar[["level_0", "name", "etiqueta", "nombre", "total"]]
tabla_tam_hogar = ids_merge(tam_hogar, estados, edades)
tabla_tam_hogar = tabla_tam_hogar[["level_0", "id", "nombre", "total"]]
tabla_tam_hogar.columns = ["id", "edad_id", "personas", "cantidad_hogares"]
tabla_tam_hogar.head(3)

,id,edad_id,personas,cantidad_hogares
0,0,180,1 persona,287663
1,1,30,1 persona,287663
2,2,40,1 persona,287663


## Nivel de educación

In [475]:
educacion1 = pd.read_csv("data_censo/educacion1.csv")
educacion2 = pd.read_csv("data_censo/educacion2.csv")

In [476]:
educacion1 = educacion1.rename(columns=name_columns)
educacion2 = educacion2.rename(columns=name_columns)

In [477]:
educacion1.iloc[:,-10:] = educacion1.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
educacion2.iloc[:,-10:] = educacion2.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))

In [478]:
educacion1["name"] = educacion1["name"].apply(limpiar_edad)
educacion2["name"] = educacion2["name"].apply(limpiar_edad)

In [479]:
male_public = educacion1[3:11]
male_private = educacion1[12:20]
male_not_enrolled = educacion1[21:29]

female_public = educacion1[31:39]
female_private = educacion1[40:48]
female_not_enrolled = educacion1[49:]

In [480]:
publica = pd.merge(male_public, female_public, on="name")
privada = pd.merge(male_private, female_private, on="name")
not_enrolled = pd.merge(male_not_enrolled, female_not_enrolled, on="name")

In [481]:
publica = uniones(publica, "Inscrito")
privada = uniones(privada, "Inscrito")
not_enrolled = uniones(not_enrolled[1:4], "No inscrito")

In [482]:
inscritos = pd.merge(publica[1:4].drop(['nombre'], axis=1), privada[1:4].drop(['nombre'], axis=1), on="name")

In [483]:
inscritos = inscritos.rename(columns={
    'Arizona_x_x': 'Arizona_x', 'Florida_x_x':'Florida_x', 'Georgia_x_x':'Georgia_x', 'Illinois_x_x':'Illinois_x',
    'Michigan_x_x':'Michigan_x', 'North Carolina_x_x':'North Carolina_x', 'Ohio_x_x':'Ohio_x',
    'Tennessee_x_x':'Tennessee_x', 'Texas_x_x':'Texas_x', 'Virginia_x_x':'Virginia_x',
    'Arizona_x_y':'Arizona_y', 'Florida_x_y':'Florida_y', 'Georgia_x_y':'Georgia_y', 'Illinois_x_y':'Illinois_y', 
    'Michigan_x_y':'Michigan_y', 'North Carolina_x_y':'North Carolina_y', 'Ohio_x_y':'Ohio_y', 'Tennessee_x_y':'Tennessee_y',
    'Texas_x_y':'Texas_y', 'Virginia_x_y':'Virginia_y'
})

In [484]:
inscritos = uniones(inscritos, "Inscritos")

rangos = ["5 to 9 years", "10 to 14 years", "15 to 17 years"]
inscritos.nombre = rangos
inscritos.name = ["Less than 9th grade", "Less than 9th grade", "9th to 12th grade, no diploma"]

not_enrolled.nombre = rangos
not_enrolled.name = ["Not enrolled", "Not enrolled", "High school graduate (includes equivalency)"]

In [485]:
inscritos = princ(inscritos)
not_enrolled = princ(not_enrolled)

In [486]:
male_18 = educacion2[3:10]
male_25 = educacion2[11:18]
male_35 = educacion2[19:26]
male_45 = educacion2[27:34]
male_65 = educacion2[35:42]

female_18 = educacion2[44:51]
female_25 = educacion2[52:59]
female_35 = educacion2[60:67]
female_45 = educacion2[68:75]
female_65 = educacion2[76:]

In [487]:
rango_18 = general("18 to 19 years", male_18, female_18) 
rango_25 = general("25 to 29 years", male_25, female_25) 
rango_35 = general("35 to 39 years", male_35, female_35)
rango_45 = general("45 to 49 years", male_45, female_45) 
rango_65 = general("65 to 69 years", male_65, female_65)

rango_20 = ultimos(rango_18, 0.65).assign(nombre="20 to 24 years")
rango_30 = ultimos(rango_25, 0.45).assign(nombre="30 to 34 years")
rango_40 = ultimos(rango_35, 0.45).assign(nombre="40 to 44 years")
rango_50 = ultimos(rango_45, 0.35).assign(nombre="50 to 54 years")
rango_55 = ultimos(rango_45, 0.20).assign(nombre="55 to 59 years")
rango_60 = ultimos(rango_45, 0.20).assign(nombre="60 to 64 years")
rango_70 = ultimos(rango_65, 0.21).assign(nombre="70 to 74 years")
rango_75 = ultimos(rango_65, 0.19).assign(nombre="75 to 79 years")
rango_80 = ultimos(rango_65, 0.17).assign(nombre="80 to 84 years")
rango_85 = ultimos(rango_65, 0.13).assign(nombre="85 years and over")

rango_18 = ultimos(rango_18, 0.45)
rango_25 = ultimos(rango_25, 0.65)
rango_35 = ultimos(rango_35, 0.65)
rango_45 = ultimos(rango_45, 0.45)
rango_65 = ultimos(rango_65, 0.30)

rangos_educacion = [inscritos, not_enrolled, rango_18, rango_20, rango_25, rango_30, rango_35, rango_40, rango_45, rango_50, rango_55, rango_60, rango_65, rango_70, rango_75, rango_80, rango_85]
nivel_educacion = pd.concat(rangos_educacion)


In [488]:
nivel_educacion = nivel_educacion.reset_index().reset_index()
nivel_educacion = nivel_educacion[["level_0", "name", "etiqueta", "nombre", "total"]]

In [489]:
nivel_educacion = nivel_educacion.rename(columns={"name": "nombre", "nombre":"name"})

In [490]:
tabla_nivel_educacion = ids_merge(nivel_educacion, estados, edades)

In [491]:
tabla_nivel_educacion = tabla_nivel_educacion[["level_0", "id", "nombre", "total"]]
tabla_nivel_educacion.columns = ["id", "edad_id", "grado", "cantidad_educacion"]
tabla_nivel_educacion.head(3)

,id,edad_id,grado,cantidad_educacion
0,0,10,Less than 9th grade,412103
1,1,20,Less than 9th grade,465369
2,2,180,"9th to 12th grade, no diploma",275584


## Ingresos

In [492]:
poblacion_ingresos = pd.read_csv("data_censo/ingresos.csv")

In [493]:
ingresos = poblacion_ingresos.iloc[1:,:]
ingresos.iloc[:,-10:] = ingresos.iloc[:,-10:].apply(lambda x: x.str.replace(',', '').astype(int))
ingresos["Label (Grouping)"] = ingresos["Label (Grouping)"].apply(limpiar_edad)
edades_ingresos = ingresos["Label (Grouping)"][ingresos["Label (Grouping)"].str.contains(r'(.*Householder.*)')].tolist()
rango_edades = [edades_ingresos[0]]*16 + [edades_ingresos[1]]*16 + [edades_ingresos[2]]*16 + [edades_ingresos[3]]*16
ingresos = ingresos.drop([1,18,35,52])
ingresos["edades"] = rango_edades
ingresos = pd.melt(ingresos, id_vars=['Label (Grouping)', 'edades'], value_vars=ingresos.columns.to_list()[1:-1], var_name='estado', value_name='cantidad')
ingresos.columns = ["ingresos","edad","estado","cantidad"]
ingresos["estado"] = ingresos["estado"].apply(nombre_estados)

In [494]:
new_rows = []
for i, row in ingresos.iterrows():
    age_list = age_range_to_list(row["edad"])
    for age in age_list:
        new_row = row.copy()
        if(type(age) is int):
            if(age == 5):
                new_row["edad"] = f"Under {age} years"
            elif(age == 85):
                new_row["edad"] = f"{age} years and over"
        else:
            new_row["edad"] = f"{age[0]} to {age[1]} years"
        new_rows.append(new_row)
tabla_ingresos = pd.DataFrame(new_rows)
tabla_ingresos = pd.merge(tabla_ingresos, estados, how="left" ,left_on='estado', right_on='estados')
tabla_ingresos = tabla_ingresos.rename(columns={"estado":"estado2",
                                  "index":"estado"})
tabla_ingresos = pd.merge(tabla_ingresos, edades, how="left", on=["edad","estado"]).reset_index()
tabla_ingresos = tabla_ingresos[["index","id","ingresos","cantidad_x"]]
tabla_ingresos.columns = ["id","edad_id","ingresos","cantidad_ingresos"]
tabla_ingresos.head(3)

,id,edad_id,ingresos,cantidad_ingresos
0,0,00,"Less than $10,000",13603
1,1,10,"Less than $10,000",13603
2,2,20,"Less than $10,000",13603


## CENSO UNIFICADO

In [498]:
censo = pd.merge(estados, edades, how="left", left_on="index", right_on="estado", suffixes=("_Estados","_Edades"))
censo = pd.merge(censo, tabla_nivel_educacion, how="inner", left_on="id", right_on="edad_id", suffixes=("","_Educacion"))
censo = pd.merge(censo, tabla_estado_civil, how="inner", left_on="id", right_on="edad_id", suffixes=("","_Civil"))
censo = pd.merge(censo, tabla_tam_hogar, how="left", left_on="id", right_on="edad_id", suffixes=("","_Hogar"))
censo = pd.merge(censo, tabla_ingresos, how="left", left_on="id", right_on="edad_id", suffixes=("","_Ingreso"))
censo = censo[['edad', 'cantidad', 
               'grado', 'cantidad_educacion',
               'estado_civil', 'cantidad_civil',
               'personas', 'cantidad_hogares',
               'ingresos', 'cantidad_ingresos',
                'estados', 'poblacion',]].reset_index()
censo = censo.rename(columns={"index":"id"})
censo

,id,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,estados,poblacion
0,0,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"Less than $10,000",13603,Arizona,7276316
1,1,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$10,000 to $14,999",6050,Arizona,7276316
2,2,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$15,000 to $19,999",5631,Arizona,7276316
3,3,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$20,000 to $24,999",9033,Arizona,7276316
4,4,18 to 19 years,109881,Less than 9th grade,3691,Soltero,189569,1 persona,287663,"$25,000 to $29,999",8104,Arizona,7276316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205435,205435,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$75,000 to $99,999",10337,Virginia,8642274
205436,205436,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$100,000 to $124,999",5873,Virginia,8642274
205437,205437,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$125,000 to $149,999",2406,Virginia,8642274
205438,205438,15 to 17 years,371036,High school graduate (includes equivalency),9415,Divorciado,206,2 o mas personas,1154204,"$150,000 to $199,999",2900,Virginia,8642274


# Perfiles

In [61]:
perfiles_edades = censo[["estados","edad","cantidad"]].drop_duplicates()
perfiles_edades['Porcentaje'] = (perfiles_edades['cantidad'] / perfiles_edades.groupby('estados')['cantidad'].transform('sum')).astype(float).round(3)
perfiles_edades['Rango'] = perfiles_edades.groupby('estados')['Porcentaje'].rank(ascending=False, method='dense')
perfiles_edades = perfiles_edades[perfiles_edades['Rango'].eq(1) | perfiles_edades.duplicated(['estados', 'Porcentaje'], keep=False)]
perfiles_edades = perfiles_edades[perfiles_edades["Rango"] == 1]
perfiles_edades = perfiles_edades.drop('Rango', axis=1)
perfiles_edades

,estados,edad,cantidad,Porcentaje
2688,Arizona,25 to 29 years,502697,0.085
32640,Florida,60 to 64 years,1509806,0.083
45120,Georgia,30 to 34 years,738064,0.085
46464,Georgia,35 to 39 years,733823,0.085
65664,Illinois,30 to 34 years,858792,0.084
67008,Illinois,35 to 39 years,864721,0.084
94272,Michigan,60 to 64 years,723695,0.088
104064,North Carolina,20 to 24 years,705531,0.082
106752,North Carolina,30 to 34 years,702577,0.082
112128,North Carolina,50 to 54 years,698866,0.082


## Perfil Arizona

In [62]:
perfil_arizona = censo[censo["estados"] == "Arizona"]
perfil_arizona = perfil_arizona[perfil_arizona["edad"] == "25 to 29 years"]
perfil_arizona["total"] = perfil_arizona["cantidad"].astype(int) + perfil_arizona["cantidad_educacion"].astype(int) + perfil_arizona["cantidad_civil"].astype(int) + perfil_arizona["cantidad_hogares"].astype(int) + perfil_arizona["cantidad_ingresos"].astype(int)
perfil_arizona['porcentaje'] = perfil_arizona['total'] /perfil_arizona['total'].sum()
perfil_arizona['rango'] = perfil_arizona['porcentaje'].rank(ascending=False, method='dense')
perfil_arizona = perfil_arizona[perfil_arizona['rango'].eq(1) | perfil_arizona.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_arizona = perfil_arizona[perfil_arizona["rango"] == 1]
perfil_arizona = perfil_arizona.drop(['rango','porcentaje'], axis=1)
perfil_arizona

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
3099,Arizona,7276316,25 to 29 years,502697,High school graduate (includes equivalency),73875,Soltero,324093,2 o mas personas,912290,"$75,000 to $99,999",145275,1958230


## Perfil Florida

In [63]:
perfil_florida = censo[censo["estados"] == "Florida"]
perfil_florida = perfil_florida[perfil_florida["edad"] == "60 to 64 years"]
perfil_florida["total"] = perfil_florida["cantidad"].astype(int) + perfil_florida["cantidad_educacion"].astype(int) + perfil_florida["cantidad_civil"].astype(int) + perfil_florida["cantidad_hogares"].astype(int) + perfil_florida["cantidad_ingresos"].astype(int)
perfil_florida['porcentaje'] = perfil_florida['total'] /perfil_florida['total'].sum()
perfil_florida['rango'] = perfil_florida['porcentaje'].rank(ascending=False, method='dense')
perfil_florida = perfil_florida[perfil_florida['rango'].eq(1) | perfil_florida.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_florida = perfil_florida[perfil_florida["rango"] == 1]
perfil_florida = perfil_florida.drop(['rango','porcentaje'], axis=1)
perfil_florida

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
33083,Florida,21781128,60 to 64 years,1509806,High school graduate (includes equivalency),22404,Casado,864393,2 o mas personas,1509617,"$75,000 to $99,999",421462,4327682


## Perfil Georgia

In [64]:
perfil_georgia = censo[censo["estados"] == "Georgia"]
perfil_georgia = perfil_georgia[(perfil_georgia["edad"] == "30 to 34 years") | (perfil_georgia["edad"] == "35 to 39 years")]
perfil_georgia["total"] = perfil_georgia["cantidad"].astype(int) + perfil_georgia["cantidad_educacion"].astype(int) + perfil_georgia["cantidad_civil"].astype(int) + perfil_georgia["cantidad_hogares"].astype(int) + perfil_georgia["cantidad_ingresos"].astype(int)
perfil_georgia['porcentaje'] = (perfil_georgia['total'] /perfil_georgia.groupby('edad')['total'].transform('sum')).astype(float).round(6)
perfil_georgia['rango'] = perfil_georgia.groupby('edad')['porcentaje'].rank(ascending=False, method='dense')
perfil_georgia = perfil_georgia[perfil_georgia['rango'].eq(1) | perfil_georgia.duplicated(['edad', 'porcentaje'], keep=False)]
perfil_georgia = perfil_georgia[perfil_georgia["rango"] == 1]
perfil_georgia = perfil_georgia.drop(['rango','porcentaje'], axis=1)
perfil_georgia

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
45531,Georgia,10799566,30 to 34 years,738064,High school graduate (includes equivalency),167755,Soltero,334186,2 o mas personas,1403391,"$75,000 to $99,999",194670,2838066
46107,Georgia,10799566,30 to 34 years,738064,Bachelor's degree,167832,Soltero,334186,2 o mas personas,1403391,"$75,000 to $99,999",194670,2838143
46907,Georgia,10799566,35 to 39 years,733823,High school graduate (includes equivalency),96896,Casado,398585,2 o mas personas,1403391,"$75,000 to $99,999",194670,2827365


## Perfil Illinois

In [65]:
perfil_illinois = censo[censo["estados"] == "Illinois"]
perfil_illinois = perfil_illinois[(perfil_illinois["edad"] == "30 to 34 years") | (perfil_illinois["edad"] == "35 to 39 years")]
perfil_illinois["total"] = perfil_illinois["cantidad"].astype(int) + perfil_illinois["cantidad_educacion"].astype(int) + perfil_illinois["cantidad_civil"].astype(int) + perfil_illinois["cantidad_hogares"].astype(int) + perfil_illinois["cantidad_ingresos"].astype(int)
perfil_illinois['porcentaje'] = (perfil_illinois['total'] /perfil_illinois.groupby('edad')['total'].transform('sum')).astype(float).round(6)
perfil_illinois['rango'] = perfil_illinois.groupby('edad')['porcentaje'].rank(ascending=False, method='dense')
perfil_illinois = perfil_illinois[perfil_illinois['rango'].eq(1) | perfil_illinois.duplicated(['edad', 'porcentaje'], keep=False)]
perfil_illinois = perfil_illinois[perfil_illinois["rango"] == 1]
perfil_illinois = perfil_illinois.drop(['rango','porcentaje'], axis=1)
perfil_illinois

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
66651,Illinois,12671469,30 to 34 years,858792,Bachelor's degree,228681,Soltero,408351,2 o mas personas,1753273,"$75,000 to $99,999",246279,3495376
68027,Illinois,12671469,35 to 39 years,864721,Bachelor's degree,123038,Casado,496845,2 o mas personas,1753273,"$75,000 to $99,999",246279,3484156


## Perfil Michigan

In [66]:
perfil_michigan = censo[censo["estados"] == "Michigan"]
perfil_michigan = perfil_michigan[perfil_michigan["edad"] == "60 to 64 years"]
perfil_michigan["total"] = perfil_michigan["cantidad"].astype(int) + perfil_michigan["cantidad_educacion"].astype(int) + perfil_michigan["cantidad_civil"].astype(int) + perfil_michigan["cantidad_hogares"].astype(int) + perfil_michigan["cantidad_ingresos"].astype(int)
perfil_michigan['porcentaje'] = (perfil_michigan['total'] /perfil_michigan['total'].sum()).astype(float).round(6)
perfil_michigan['rango'] = perfil_michigan['porcentaje'].rank(ascending=False, method='dense')
perfil_michigan = perfil_michigan[perfil_michigan['rango'].eq(1) | perfil_michigan.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_michigan = perfil_michigan[perfil_michigan["rango"] == 1]
perfil_michigan = perfil_michigan.drop(['rango','porcentaje'], axis=1)
perfil_michigan

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
94715,Michigan,10050811,60 to 64 years,723695,High school graduate (includes equivalency),10621,Casado,443712,2 o mas personas,751468,"$75,000 to $99,999",197063,2126559


## Perfil North Carolina

In [67]:
perfil_carolina = censo[censo["estados"] == "North Carolina"]
perfil_carolina = perfil_carolina[(perfil_carolina["edad"] == "20 to 24 years") | (perfil_carolina["edad"] == "30 to 34 years") | (perfil_carolina["edad"] == "50 to 54 years")]
perfil_carolina["total"] = perfil_carolina["cantidad"].astype(int) + perfil_carolina["cantidad_educacion"].astype(int) + perfil_carolina["cantidad_civil"].astype(int) + perfil_carolina["cantidad_hogares"].astype(int) + perfil_carolina["cantidad_ingresos"].astype(int)
perfil_carolina['porcentaje'] = (perfil_carolina['total'] /perfil_carolina.groupby('edad')['total'].transform('sum')).astype(float).round(6)
perfil_carolina['rango'] = perfil_carolina.groupby('edad')['porcentaje'].rank(ascending=False, method='dense')
perfil_carolina = perfil_carolina[perfil_carolina['rango'].eq(1) | perfil_carolina.duplicated(['edad', 'porcentaje'], keep=False)]
perfil_carolina = perfil_carolina[perfil_carolina["rango"] == 1]
perfil_carolina = perfil_carolina.drop(['rango','porcentaje'], axis=1)
perfil_carolina

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
104656,North Carolina,10551162,20 to 24 years,705531,"Some college, no degree",238635,Soltero,616977,2 o mas personas,1406562,"Less than $10,000",31793,2999498
107771,North Carolina,10551162,30 to 34 years,702577,Bachelor's degree,170025,Casado,318210,2 o mas personas,1406562,"$75,000 to $99,999",192070,2789444
112571,North Carolina,10551162,50 to 54 years,698866,High school graduate (includes equivalency),236574,Casado,428246,2 o mas personas,1406562,"$75,000 to $99,999",203699,2973947


## Perfil Ohio

In [68]:
perfil_ohio = censo[censo["estados"] == "Ohio"]
perfil_ohio = perfil_ohio[perfil_ohio["edad"] == "60 to 64 years"]
perfil_ohio["total"] = perfil_ohio["cantidad"].astype(int) + perfil_ohio["cantidad_educacion"].astype(int) + perfil_ohio["cantidad_civil"].astype(int) + perfil_ohio["cantidad_hogares"].astype(int) + perfil_ohio["cantidad_ingresos"].astype(int)
perfil_ohio['porcentaje'] = (perfil_ohio['total'] /perfil_ohio['total'].sum()).astype(float).round(6)
perfil_ohio['rango'] = perfil_ohio['porcentaje'].rank(ascending=False, method='dense')
perfil_ohio = perfil_ohio[perfil_ohio['rango'].eq(1) | perfil_ohio.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_ohio = perfil_ohio[perfil_ohio["rango"] == 1]
perfil_ohio = perfil_ohio.drop(['rango','porcentaje'], axis=1)
perfil_ohio

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
135803,Ohio,11780017,60 to 64 years,823595,High school graduate (includes equivalency),14196,Casado,488219,2 o mas personas,902380,"$75,000 to $99,999",243810,2472200


## Perfil Tennessee

In [69]:
perfil_tennessee = censo[censo["estados"] == "Tennessee"]
perfil_tennessee = perfil_tennessee[perfil_tennessee["edad"] == "25 to 29 years"]
perfil_tennessee["total"] = perfil_tennessee["cantidad"].astype(int) + perfil_tennessee["cantidad_educacion"].astype(int) + perfil_tennessee["cantidad_civil"].astype(int) + perfil_tennessee["cantidad_hogares"].astype(int) + perfil_tennessee["cantidad_ingresos"].astype(int)
perfil_tennessee['porcentaje'] = (perfil_tennessee['total'] /perfil_tennessee['total'].sum()).astype(float).round(6)
perfil_tennessee['rango'] = perfil_tennessee['porcentaje'].rank(ascending=False, method='dense')
perfil_tennessee = perfil_tennessee[perfil_tennessee['rango'].eq(1) | perfil_tennessee.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_tennessee = perfil_tennessee[perfil_tennessee["rango"] == 1]
perfil_tennessee = perfil_tennessee.drop(['rango','porcentaje'], axis=1)
perfil_tennessee

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
146907,Tennessee,6975218,25 to 29 years,474393,High school graduate (includes equivalency),78697,Soltero,279968,2 o mas personas,920025,"$75,000 to $99,999",139346,1892429


## Perfil Texas

In [70]:
perfil_texas = censo[censo["estados"] == "Texas"]
perfil_texas = perfil_texas[perfil_texas["edad"] == "30 to 34 years"]
perfil_texas["total"] = perfil_texas["cantidad"].astype(int) + perfil_texas["cantidad_educacion"].astype(int) + perfil_texas["cantidad_civil"].astype(int) + perfil_texas["cantidad_hogares"].astype(int) + perfil_texas["cantidad_ingresos"].astype(int)
perfil_texas['porcentaje'] = (perfil_texas['total'] /perfil_texas['total'].sum()).astype(float).round(6)
perfil_texas['rango'] = perfil_texas['porcentaje'].rank(ascending=False, method='dense')
perfil_texas = perfil_texas[perfil_texas['rango'].eq(1) | perfil_texas.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_texas = perfil_texas[perfil_texas["rango"] == 1]
perfil_texas = perfil_texas.drop(['rango','porcentaje'], axis=1)
perfil_texas

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
168827,Texas,29527941,30 to 34 years,2158231,High school graduate (includes equivalency),479449,Casado,1055917,2 o mas personas,4045349,"$75,000 to $99,999",550728,8289674
169403,Texas,29527941,30 to 34 years,2158231,Bachelor's degree,481430,Casado,1055917,2 o mas personas,4045349,"$75,000 to $99,999",550728,8291655


## Perfil Virginia

In [71]:
perfil_virginia = censo[censo["estados"] == "Virginia"]
perfil_virginia = perfil_virginia[perfil_virginia["edad"] == "35 to 39 years"]
perfil_virginia["total"] = perfil_virginia["cantidad"].astype(int) + perfil_virginia["cantidad_educacion"].astype(int) + perfil_virginia["cantidad_civil"].astype(int) + perfil_virginia["cantidad_hogares"].astype(int) + perfil_virginia["cantidad_ingresos"].astype(int)
perfil_virginia['porcentaje'] = (perfil_virginia['total'] /perfil_virginia['total'].sum()).astype(float).round(6)
perfil_virginia['rango'] = perfil_virginia['porcentaje'].rank(ascending=False, method='dense')
perfil_virginia = perfil_virginia[perfil_virginia['rango'].eq(1) | perfil_virginia.duplicated(['estados', 'porcentaje'], keep=False)]
perfil_virginia = perfil_virginia[perfil_virginia["rango"] == 1]
perfil_virginia = perfil_virginia.drop(['rango','porcentaje'], axis=1)
perfil_virginia

,estados,poblacion,edad,cantidad,grado,cantidad_educacion,estado_civil,cantidad_civil,personas,cantidad_hogares,ingresos,cantidad_ingresos,total
191291,Virginia,8642274,35 to 39 years,600446,Bachelor's degree,86465,Casado,345506,2 o mas personas,1154204,"$75,000 to $99,999",155625,2342246


## Perfiles unificados

In [77]:
perfiles = pd.concat([perfil_arizona,
           perfil_florida,
           perfil_georgia,
           perfil_illinois, 
           perfil_michigan, 
           perfil_carolina, 
           perfil_ohio, 
           perfil_tennessee, 
           perfil_texas, 
           perfil_virginia])
perfiles = perfiles[['estados', 'edad', 'grado', 'estado_civil', 'personas', 'ingresos']].reset_index().reset_index()
perfiles = perfiles.rename(columns={'index':'id_censo', 'level_0':'id'})
perfiles

,id,id_censo,estados,edad,grado,estado_civil,personas,ingresos
0,0,3099,Arizona,25 to 29 years,High school graduate (includes equivalency),Soltero,2 o mas personas,"$75,000 to $99,999"
1,1,33083,Florida,60 to 64 years,High school graduate (includes equivalency),Casado,2 o mas personas,"$75,000 to $99,999"
2,2,45531,Georgia,30 to 34 years,High school graduate (includes equivalency),Soltero,2 o mas personas,"$75,000 to $99,999"
3,3,46107,Georgia,30 to 34 years,Bachelor's degree,Soltero,2 o mas personas,"$75,000 to $99,999"
4,4,46907,Georgia,35 to 39 years,High school graduate (includes equivalency),Casado,2 o mas personas,"$75,000 to $99,999"
5,5,66651,Illinois,30 to 34 years,Bachelor's degree,Soltero,2 o mas personas,"$75,000 to $99,999"
6,6,68027,Illinois,35 to 39 years,Bachelor's degree,Casado,2 o mas personas,"$75,000 to $99,999"
7,7,94715,Michigan,60 to 64 years,High school graduate (includes equivalency),Casado,2 o mas personas,"$75,000 to $99,999"
8,8,104656,North Carolina,20 to 24 years,"Some college, no degree",Soltero,2 o mas personas,"Less than $10,000"
9,9,107771,North Carolina,30 to 34 years,Bachelor's degree,Casado,2 o mas personas,"$75,000 to $99,999"


In [413]:
perfiles.dtypes

id               int64
id_censo         int64
estados         object
edad            object
grado           object
estado_civil    object
personas        object
ingresos        object
dtype: object

# Planes

Condiciones:
- Estado
- Edad
- Tasa individual
    - Arizona = 16%
    - Florida = 20%
    - Georgia = 17%
    - Illinois = 17%
    - Michigan = 19%
    - North Carolina = 18%
    - Ohio = 20%
    - Tennessee = 20%
    - Texas = 16%
    - Virginia = 16%
- Tipo de plan:
    - [20 to 24 years, 25 to 29 years] [High school graduate, Some college] = HMO
    - [30 to 34 years, 35 to 39 years] [High school graduate]  = HMO
    - [30 to 34 years, 35 to 39 years] [Bachelor's degree] = EPO
    - [50 to 54 years] [High school graduate] = POS
    - [60 to 64 years] [High school graduate] = PPO
- Nivel de Metal
    - [20 to 24 years] [Soltero] [Less than $10,000] = Catastrophic
    - [25 to 29 years] [Soltero] [$75,000 to $99,999] = Bronze
    - [30 to 34 years] [Soltero] [$75,000 to $99,999] = Bronce Ampliado
    - [30 to 34 years] [Casados] [$75,000 to $99,999] = Plata
    - [35 to 39 years] [Casados] [$75,000 to $99,999] = Oro
    - [50 to 54 years] [Casados] [$75,000 to $99,999] = Plata
    - [60 to 64 years] [Casados] [$75,000 to $99,999] = Bronce Ampliado

- Referencias: 
    - https://www.sofi.com/learn/content/average-salary-in-us/
    - https://www.forbes.com/advisor/health-insurance/most-and-least-expensive-states-for-health-care-ranked/

- tipo
    - EPO    686.971327
    - HMO    592.374838
    - POS    711.208490
    - PPO    738.629254

- nivel_metal
    - Bronze              482.983387
    - Catastrophic        379.872661
    - Expanded Bronze     535.799815
    - Gold                703.640874
    - Platinum           1087.302245    
    - Silver              653.467869

## Planes perfiles

In [110]:
tabla_planes = pd.read_csv("attributes.csv")
tabla_transparency = pd.read_excel('transparency.xlsx', sheet_name = 'Transparency 2023 - Ind QHP', header = 2, usecols = 'A:AD')

In [111]:
planes = tabla_planes[tabla_planes["DentalOnlyPlan"]=="No"]
planes = planes[["StandardComponentId","StateCode","IssuerId","PlanMarketingName","PlanType","MetalLevel"]]
planes = planes[planes["StateCode"].isin(state_filter)]
planes = planes.drop_duplicates()
planes = planes.rename(columns={"IssuerId":"Issuer_ID"})

In [112]:
transparency = tabla_transparency.iloc[:,1:3]
transparency = transparency.drop_duplicates()

In [113]:
planes = pd.merge(planes, transparency, how="inner" ,on=["Issuer_ID"])
planes = planes[["StandardComponentId","StateCode","Issuer_Name","PlanMarketingName","PlanType","MetalLevel"]]
planes["StateCode"] = planes["StateCode"].apply(codigo_a_estado)
planes = planes[["StandardComponentId",'StateCode',"PlanMarketingName","PlanType","MetalLevel","Issuer_Name"]]
planes.columns = ["id","estado_id","nombre_plan","tipo","nivel_metal","nombre_proveedor"]

In [114]:
planes.head(3)

,id,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor
0,13877AZ0070002,Arizona,Bronze Classic- PCP Saver,HMO,Expanded Bronze,"Oscar Health Plan, Inc."
1,13877AZ0070003,Arizona,Bronze Classic,HMO,Expanded Bronze,"Oscar Health Plan, Inc."
2,13877AZ0070005,Arizona,Bronze Elite- $0 Ded+PCP Saver,HMO,Expanded Bronze,"Oscar Health Plan, Inc."


## Edad planes

In [115]:
edades_seguros = pd.read_csv("Rate.csv")

In [232]:
edades_seguros_tabla = edades_seguros[edades_seguros["StateCode"].isin(state_filter)]
edades_seguros_tabla["Couple"] = edades_seguros_tabla["Couple"].fillna("0")
edades_seguros_tabla["PrimarySubscriberAndOneDependent"] = edades_seguros_tabla["PrimarySubscriberAndOneDependent"].fillna("0")
edades_seguros_tabla["PrimarySubscriberAndTwoDependents"] = edades_seguros_tabla["PrimarySubscriberAndTwoDependents"].fillna("0")
edades_seguros_tabla["PrimarySubscriberAndThreeOrMoreDependents"] = edades_seguros_tabla["PrimarySubscriberAndThreeOrMoreDependents"].fillna("0")
edades_seguros_tabla["CoupleAndOneDependent"] = edades_seguros_tabla["CoupleAndOneDependent"].fillna("0")
edades_seguros_tabla["CoupleAndTwoDependents"] = edades_seguros_tabla["CoupleAndTwoDependents"].fillna("0")
edades_seguros_tabla["CoupleAndThreeOrMoreDependents"] = edades_seguros_tabla["CoupleAndThreeOrMoreDependents"].fillna("0")
edades_seguros_tabla = edades_seguros_tabla.drop(["IndividualTobaccoRate","Tobacco"], axis=1)

In [233]:
edades_seguros_tabla.head(3)

,BusinessYear,StateCode,IssuerId,SourceName,ImportDate,RateEffectiveDate,RateExpirationDate,PlanId,RatingAreaId,Age,IndividualRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents
64039,2023,AZ,12303,HIOS,2022-06-14 01:01:37,2023-01-01,2023-12-31,12303AZ0010001,Rating Area 1,0-14,29.78,0,0,0,0,0,0,0
64040,2023,AZ,12303,HIOS,2022-06-14 01:01:37,2023-01-01,2023-12-31,12303AZ0010001,Rating Area 2,0-14,29.78,0,0,0,0,0,0,0
64041,2023,AZ,12303,HIOS,2022-06-14 01:01:37,2023-01-01,2023-12-31,12303AZ0010001,Rating Area 3,0-14,29.78,0,0,0,0,0,0,0


## Tabla general planes

In [236]:
tabla_general_planes = pd.merge(planes, edades_seguros_tabla, how="left", left_on="id", right_on="PlanId")

In [237]:
tabla_general_planes.head(3)

,id,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,IssuerId,SourceName,...,RatingAreaId,Age,IndividualRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents
0,13877AZ0070002,Arizona,Bronze Classic- PCP Saver,HMO,Expanded Bronze,"Oscar Health Plan, Inc.",2023,AZ,13877,HIOS,...,Rating Area 4,0-14,200.78,0,0,0,0,0,0,0
1,13877AZ0070002,Arizona,Bronze Classic- PCP Saver,HMO,Expanded Bronze,"Oscar Health Plan, Inc.",2023,AZ,13877,HIOS,...,Rating Area 4,15,218.63,0,0,0,0,0,0,0
2,13877AZ0070002,Arizona,Bronze Classic- PCP Saver,HMO,Expanded Bronze,"Oscar Health Plan, Inc.",2023,AZ,13877,HIOS,...,Rating Area 4,16,225.45,0,0,0,0,0,0,0


## Plan para perfil Arizona

Perfil
- 25 to 29 years
- High school graduate (includes equivalency)	
- Soltero
- 2 o mas personas
- $75,000 to $99,999

In [338]:
plan_arizona = tabla_general_planes[tabla_general_planes["estado_id"] == "Arizona"]
plan_arizona = plan_arizona[plan_arizona["Age"].isin(['25','26','27','28','29'])]
plan_arizona = plan_arizona[plan_arizona["IndividualRate"] < (75000*0.16)]
plan_arizona = plan_arizona[plan_arizona["tipo"] == "HMO"]
plan_arizona = plan_arizona[plan_arizona["nivel_metal"] == "Bronze"]
plan_arizona_perfil = plan_arizona.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_arizona_perfil = plan_arizona_perfil.loc[plan_arizona_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_arizona = pd.merge(plan_arizona_perfil, plan_arizona, how="left", on=["id","Age","IndividualRate"])
plan_arizona["id_perfil"] = 0
plan_arizona

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,40702AZ0060014,25,227.59,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
1,40702AZ0060014,26,232.12,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
2,40702AZ0060014,27,237.56,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
3,40702AZ0060014,28,246.41,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
4,40702AZ0060014,29,253.66,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0


## Plan para perfil Florida

Perfil
- 60 to 64 years
- High school graduate (includes equivalency)	
- Casado
- 2 o mas personas
- $75,000 to $99,999

In [342]:
plan_florida = tabla_general_planes[tabla_general_planes["estado_id"] == "Florida"]
plan_florida = plan_florida[plan_florida["Age"].isin(['60','61','62','63','64'])]
plan_florida = plan_florida[plan_florida["IndividualRate"] < (75000*0.20)]
#plan_florida = plan_florida[plan_florida["tipo"] == "PPO"]
plan_florida = plan_florida[plan_florida["nivel_metal"] == "Expanded Bronze"]
plan_florida_perfil = plan_florida.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_florida_perfil = plan_florida_perfil.loc[plan_florida_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_florida = pd.merge(plan_florida_perfil, plan_florida, how="left", on=["id","Age","IndividualRate"])
plan_florida["id_perfil"] = 1
plan_florida

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,40572FL0070003,60,715.05,Florida,Bronze Classic,EPO,Expanded Bronze,Oscar Insurance Company of Florida,2023,FL,...,40572FL0070003,Rating Area 41,0,0,0,0,0,0,0,1
1,40572FL0070003,61,740.35,Florida,Bronze Classic,EPO,Expanded Bronze,Oscar Insurance Company of Florida,2023,FL,...,40572FL0070003,Rating Area 41,0,0,0,0,0,0,0,1
2,40572FL0070003,62,756.94,Florida,Bronze Classic,EPO,Expanded Bronze,Oscar Insurance Company of Florida,2023,FL,...,40572FL0070003,Rating Area 41,0,0,0,0,0,0,0,1
3,40572FL0070003,63,777.76,Florida,Bronze Classic,EPO,Expanded Bronze,Oscar Insurance Company of Florida,2023,FL,...,40572FL0070003,Rating Area 41,0,0,0,0,0,0,0,1


## Plan para perfil Georgia

Perfil
- 30 to 34 years - 30 to 34 years - 35 to 39 years
- High school graduate  - Bachelor's degree - High school graduate
- Soltero - Soltero - Casado
- 2 o mas personas - 2 o mas personas - 2 o mas personas
- $75,000 to $99,999 - $75,000 to $99,999 - $75,000 to $99,999

In [297]:
plan_georgia = tabla_general_planes[tabla_general_planes["estado_id"] == "Georgia"]
plan_georgia = plan_georgia[plan_georgia["Age"].isin(['30','31','32','33','34'])]
plan_georgia = plan_georgia[plan_georgia["IndividualRate"] < (75000*0.17)]
plan_georgia = plan_georgia[plan_georgia["tipo"] == "HMO"]
plan_georgia = plan_georgia[plan_georgia["nivel_metal"] == "Expanded Bronze"]
plan_georgia_perfil = plan_georgia.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_georgia_perfil = plan_georgia_perfil.loc[plan_georgia_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_georgia = pd.merge(plan_georgia_perfil, plan_georgia, how="left", on=["id","Age","IndividualRate"])
plan_georgia["id_perfil"] = 2
plan_georgia.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,83761GA0110027,30,224.86,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,2
1,83761GA0110027,31,229.61,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,2
2,83761GA0110027,32,234.37,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,2


In [298]:
plan_georgia1 = tabla_general_planes[tabla_general_planes["estado_id"] == "Georgia"]
plan_georgia1 = plan_georgia1[plan_georgia1["Age"].isin(['30','31','32','33','34'])]
plan_georgia1 = plan_georgia1[plan_georgia1["IndividualRate"] < (75000*0.17)]
#plan_georgia1 = plan_georgia1[plan_georgia1["tipo"] == "EPO"]
plan_georgia1 = plan_georgia1[plan_georgia1["nivel_metal"] == "Expanded Bronze"]
plan_georgia1_perfil = plan_georgia1.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_georgia1_perfil = plan_georgia1_perfil.loc[plan_georgia1_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_georgia1 = pd.merge(plan_georgia1_perfil, plan_georgia1, how="left", on=["id","Age","IndividualRate"])
plan_georgia1["id_perfil"] = 3
plan_georgia1

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,83761GA0110027,30,224.86,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,3
1,83761GA0110027,31,229.61,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,3
2,83761GA0110027,32,234.37,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,3
3,83761GA0110027,33,237.34,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,3
4,83761GA0110027,34,240.51,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,3


In [299]:
plan_georgia2 = tabla_general_planes[tabla_general_planes["estado_id"] == "Georgia"]
plan_georgia2 = plan_georgia2[plan_georgia2["Age"].isin(['35','36','37','38','39'])]
plan_georgia2 = plan_georgia2[plan_georgia2["IndividualRate"] < (75000*0.17)]
plan_georgia2 = plan_georgia2[plan_georgia2["tipo"] == "HMO"]
plan_georgia2 = plan_georgia2[plan_georgia2["nivel_metal"] == "Gold"]
plan_georgia2_perfil = plan_georgia2.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_georgia2_perfil = plan_georgia2_perfil.loc[plan_georgia2_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_georgia2 = pd.merge(plan_georgia2_perfil, plan_georgia2, how="left", on=["id","Age","IndividualRate"])
plan_georgia2["id_perfil"] = 4
plan_georgia2

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,70893GA0010073,35,316.08,Georgia,Clear Gold,HMO,Gold,Ambetter of Peach State Inc.,2023,GA,...,70893GA0010073,Rating Area 10,0,0,0,0,0,0,0,4
1,70893GA0010073,36,318.15,Georgia,Clear Gold,HMO,Gold,Ambetter of Peach State Inc.,2023,GA,...,70893GA0010073,Rating Area 10,0,0,0,0,0,0,0,4
2,70893GA0010073,37,320.22,Georgia,Clear Gold,HMO,Gold,Ambetter of Peach State Inc.,2023,GA,...,70893GA0010073,Rating Area 10,0,0,0,0,0,0,0,4
3,70893GA0010073,38,322.29,Georgia,Clear Gold,HMO,Gold,Ambetter of Peach State Inc.,2023,GA,...,70893GA0010073,Rating Area 10,0,0,0,0,0,0,0,4
4,70893GA0010073,39,326.43,Georgia,Clear Gold,HMO,Gold,Ambetter of Peach State Inc.,2023,GA,...,70893GA0010073,Rating Area 10,0,0,0,0,0,0,0,4


In [301]:
plan_georgia_general = pd.concat([plan_georgia,plan_georgia1,plan_georgia2])
#plan_georgia_general = plan_georgia_general.drop(["index"], axis=1)
#plan_georgia_general = plan_georgia_general.rename(columns={"id":"id_plan","level_0":"id"})
plan_georgia_general.head(2)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,83761GA0110027,30,224.86,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,2
1,83761GA0110027,31,229.61,Georgia,SoloCare HMO Expanded Bronze Standardized,HMO,Expanded Bronze,Alliant Health Plans,2023,GA,...,83761GA0110027,Rating Area 11,0,0,0,0,0,0,0,2


## Plan para perfil Illinois

Perfil
- 30 to 34 years - 35 to 39 years
- Bachelor's degree  - Bachelor's degree
- Soltero - Casado
- 2 o mas personas - 2 o mas personas
- $75,000 to $99,999 - $75,000 to $99,999

In [302]:
plan_illinois = tabla_general_planes[tabla_general_planes["estado_id"] == "Illinois"]
plan_illinois = plan_illinois[plan_illinois["Age"].isin(['30','31','32','33','34'])]
plan_illinois = plan_illinois[plan_illinois["IndividualRate"] < (75000*0.17)]
#plan_illinois = plan_illinois[plan_illinois["tipo"] == "EPO"]
plan_illinois = plan_illinois[plan_illinois["nivel_metal"] == "Expanded Bronze"]
plan_illinois_perfil = plan_illinois.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_illinois_perfil = plan_illinois_perfil.loc[plan_illinois_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_illinois = pd.merge(plan_illinois_perfil, plan_illinois, how="left", on=["id","Age","IndividualRate"])
plan_illinois["id_perfil"] = 5
plan_illinois.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,36096IL1000009,30,261.53,Illinois,Blue FocusCare Bronze℠ 209,HMO,Expanded Bronze,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000009,Rating Area 1,0,0,0,0,0,0,0,5
1,36096IL1000009,31,267.06,Illinois,Blue FocusCare Bronze℠ 209,HMO,Expanded Bronze,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000009,Rating Area 1,0,0,0,0,0,0,0,5
2,36096IL1000009,32,272.59,Illinois,Blue FocusCare Bronze℠ 209,HMO,Expanded Bronze,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000009,Rating Area 1,0,0,0,0,0,0,0,5


In [303]:
plan_illinois2 = tabla_general_planes[tabla_general_planes["estado_id"] == "Illinois"]
plan_illinois2 = plan_illinois2[plan_illinois2["Age"].isin(['35','36','37','38','39'])]
plan_illinois2 = plan_illinois2[plan_illinois2["IndividualRate"] < (75000*0.17)]
#plan_illinois2 = plan_illinois2[plan_illinois2["tipo"] == "EPO"]
plan_illinois2 = plan_illinois2[plan_illinois2["nivel_metal"] == "Silver"]
plan_illinois2_perfil = plan_illinois2.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_illinois2_perfil = plan_illinois2_perfil.loc[plan_illinois2_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_illinois2 = pd.merge(plan_illinois2_perfil, plan_illinois2, how="left", on=["id","Age","IndividualRate"])
plan_illinois2["id_perfil"] = 6
plan_illinois2.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,36096IL1000008,35,358.87,Illinois,Blue FocusCare Silver℠ 210,HMO,Silver,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000008,Rating Area 1,0,0,0,0,0,0,0,6
1,36096IL1000008,36,361.22,Illinois,Blue FocusCare Silver℠ 210,HMO,Silver,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000008,Rating Area 1,0,0,0,0,0,0,0,6
2,36096IL1000008,37,363.57,Illinois,Blue FocusCare Silver℠ 210,HMO,Silver,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000008,Rating Area 1,0,0,0,0,0,0,0,6


In [305]:
plan_illinois_general = pd.concat([plan_illinois,plan_illinois2])
#plan_illinois_general = plan_illinois_general.drop(["index"], axis=1)
#plan_illinois_general = plan_illinois_general.rename(columns={"id":"id_plan","level_0":"id"})
plan_illinois_general.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,36096IL1000009,30,261.53,Illinois,Blue FocusCare Bronze℠ 209,HMO,Expanded Bronze,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000009,Rating Area 1,0,0,0,0,0,0,0,5
1,36096IL1000009,31,267.06,Illinois,Blue FocusCare Bronze℠ 209,HMO,Expanded Bronze,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000009,Rating Area 1,0,0,0,0,0,0,0,5
2,36096IL1000009,32,272.59,Illinois,Blue FocusCare Bronze℠ 209,HMO,Expanded Bronze,Blue Cross Blue Shield of Illinois,2023,IL,...,36096IL1000009,Rating Area 1,0,0,0,0,0,0,0,5


## Plan para perfil Michigan

Perfil
- 60 to 64 years
- High school graduate (includes equivalency)	
- Casado
- 2 o mas personas
- $75,000 to $99,999

In [306]:
plan_michigan = tabla_general_planes[tabla_general_planes["estado_id"] == "Michigan"]
plan_michigan = plan_michigan[plan_michigan["Age"].isin(['60','61','62','63','64'])]
plan_michigan = plan_michigan[plan_michigan["IndividualRate"] < (75000*0.19)]
plan_michigan = plan_michigan[plan_michigan["tipo"] == "PPO"]
plan_michigan = plan_michigan[plan_michigan["nivel_metal"] == "Expanded Bronze"]
plan_michigan_perfil = plan_michigan.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_michigan_perfil = plan_michigan_perfil.loc[plan_michigan_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_michigan = pd.merge(plan_michigan_perfil, plan_michigan, how="left", on=["id","Age","IndividualRate"])
plan_michigan["id_perfil"] = 7
plan_michigan.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,15560MI0350002,60,744.50,Michigan,Blue Cross® Premier PPO Bronze HSA,PPO,Expanded Bronze,Blue Cross Blue Shield of Michigan Mutual Insu...,2023,MI,...,15560MI0350002,Rating Area 3,0,0,0,0,0,0,0,7
1,15560MI0350002,61,770.84,Michigan,Blue Cross® Premier PPO Bronze HSA,PPO,Expanded Bronze,Blue Cross Blue Shield of Michigan Mutual Insu...,2023,MI,...,15560MI0350002,Rating Area 3,0,0,0,0,0,0,0,7
2,15560MI0350002,62,788.12,Michigan,Blue Cross® Premier PPO Bronze HSA,PPO,Expanded Bronze,Blue Cross Blue Shield of Michigan Mutual Insu...,2023,MI,...,15560MI0350002,Rating Area 3,0,0,0,0,0,0,0,7


## Plan para perfil Carolina del Norte

Perfil
- 20 to 24 years - 30 to 34 years - 50 to 54 years
- Some college  - Bachelor's degree - High school graduate
- Soltero - Casado - Casado
- 2 o mas personas - 2 o mas personas - 2 o mas personas
- Less than $10,000 - $75,000 to $99,999 - $75,000 to $99,999

In [307]:
plan_carolina = tabla_general_planes[tabla_general_planes["estado_id"] == "North Carolina"]
plan_carolina = plan_carolina[plan_carolina["Age"].isin(['20','21','22','23','24'])]
plan_carolina = plan_carolina[plan_carolina["IndividualRate"] < (75000*0.18)]
plan_carolina = plan_carolina[plan_carolina["tipo"] == "HMO"]
plan_carolina = plan_carolina[plan_carolina["nivel_metal"] == "Catastrophic"]
plan_carolina_perfil = plan_carolina.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_carolina_perfil = plan_carolina_perfil.loc[plan_carolina_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_carolina = pd.merge(plan_carolina_perfil, plan_carolina, how="left", on=["id","Age","IndividualRate"])
plan_carolina["id_perfil"] = 8
plan_carolina.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,77320NC0010001,20,182.31,North Carolina,Friday Catastrophic,HMO,Catastrophic,Friday Health Plans of North Carolina Inc,2023,NC,...,77320NC0010001,Rating Area 7,0,0,0,0,0,0,0,8
1,77320NC0010001,21,187.95,North Carolina,Friday Catastrophic,HMO,Catastrophic,Friday Health Plans of North Carolina Inc,2023,NC,...,77320NC0010001,Rating Area 7,0,0,0,0,0,0,0,8
2,77320NC0010001,22,187.95,North Carolina,Friday Catastrophic,HMO,Catastrophic,Friday Health Plans of North Carolina Inc,2023,NC,...,77320NC0010001,Rating Area 7,0,0,0,0,0,0,0,8


In [308]:
plan_carolina1 = tabla_general_planes[tabla_general_planes["estado_id"] == "North Carolina"]
plan_carolina1 = plan_carolina1[plan_carolina1["Age"].isin(['30','31','32','33','34'])]
plan_carolina1 = plan_carolina1[plan_carolina1["IndividualRate"] < (75000*0.18)]
plan_carolina1 = plan_carolina1[plan_carolina1["tipo"] == "EPO"]
plan_carolina1 = plan_carolina1[plan_carolina1["nivel_metal"] == "Silver"]
plan_carolina1_perfil = plan_carolina1.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_carolina1_perfil = plan_carolina1_perfil.loc[plan_carolina1_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_carolina1 = pd.merge(plan_carolina1_perfil, plan_carolina1, how="left", on=["id","Age","IndividualRate"])
plan_carolina1["id_perfil"] = 9
plan_carolina1.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,11512NC0420013,30,391.03,North Carolina,Blue Local Silver Preferred 3100 | 3 Free PCP ...,EPO,Silver,Blue Cross and Blue Shield of NC,2023,NC,...,11512NC0420013,Rating Area 6,0,0,0,0,0,0,0,9
1,11512NC0420013,31,399.30,North Carolina,Blue Local Silver Preferred 3100 | 3 Free PCP ...,EPO,Silver,Blue Cross and Blue Shield of NC,2023,NC,...,11512NC0420013,Rating Area 6,0,0,0,0,0,0,0,9
2,11512NC0420013,32,407.57,North Carolina,Blue Local Silver Preferred 3100 | 3 Free PCP ...,EPO,Silver,Blue Cross and Blue Shield of NC,2023,NC,...,11512NC0420013,Rating Area 6,0,0,0,0,0,0,0,9


In [309]:
plan_carolina2 = tabla_general_planes[tabla_general_planes["estado_id"] == "North Carolina"]
plan_carolina2 = plan_carolina2[plan_carolina2["Age"].isin(['50','51','52','53','54'])]
plan_carolina2 = plan_carolina2[plan_carolina2["IndividualRate"] < (75000*0.18)]
plan_carolina2 = plan_carolina2[plan_carolina2["tipo"] == "POS"]
plan_carolina2 = plan_carolina2[plan_carolina2["nivel_metal"] == "Silver"]
plan_carolina2_perfil = plan_carolina2.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_carolina2_perfil = plan_carolina2_perfil.loc[plan_carolina2_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_carolina2 = pd.merge(plan_carolina2_perfil, plan_carolina2, how="left", on=["id","Age","IndividualRate"])
plan_carolina2["id_perfil"] = 10
plan_carolina2.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,11512NC0100064,50,801.68,North Carolina,Blue Value Silver Preferred 3100 | 3 Free PCP...,POS,Silver,Blue Cross and Blue Shield of NC,2023,NC,...,11512NC0100064,Rating Area 6,0,0,0,0,0,0,0,10
1,11512NC0100064,51,837.14,North Carolina,Blue Value Silver Preferred 3100 | 3 Free PCP...,POS,Silver,Blue Cross and Blue Shield of NC,2023,NC,...,11512NC0100064,Rating Area 6,0,0,0,0,0,0,0,10
2,11512NC0100064,52,876.19,North Carolina,Blue Value Silver Preferred 3100 | 3 Free PCP...,POS,Silver,Blue Cross and Blue Shield of NC,2023,NC,...,11512NC0100064,Rating Area 6,0,0,0,0,0,0,0,10


In [310]:
plan_carolina_general  = pd.concat([plan_carolina,plan_carolina1,plan_carolina2])
#plan_carolina_general  = plan_carolina_general .drop(["index"], axis=1)
#plan_carolina_general  = plan_carolina_general .rename(columns={"id":"id_plan","level_0":"id"})
plan_carolina_general.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,77320NC0010001,20,182.31,North Carolina,Friday Catastrophic,HMO,Catastrophic,Friday Health Plans of North Carolina Inc,2023,NC,...,77320NC0010001,Rating Area 7,0,0,0,0,0,0,0,8
1,77320NC0010001,21,187.95,North Carolina,Friday Catastrophic,HMO,Catastrophic,Friday Health Plans of North Carolina Inc,2023,NC,...,77320NC0010001,Rating Area 7,0,0,0,0,0,0,0,8
2,77320NC0010001,22,187.95,North Carolina,Friday Catastrophic,HMO,Catastrophic,Friday Health Plans of North Carolina Inc,2023,NC,...,77320NC0010001,Rating Area 7,0,0,0,0,0,0,0,8


## Plan para perfil Ohio

Perfil
- 60 to 64 years
- High school graduate (includes equivalency)	
- Casado
- 2 o mas personas
- $75,000 to $99,999

In [311]:
plan_ohio = tabla_general_planes[tabla_general_planes["estado_id"] == "Ohio"]
plan_ohio = plan_ohio[plan_ohio["Age"].isin(['60','61','62','63','64'])]
plan_ohio = plan_ohio[plan_ohio["IndividualRate"] < (75000*0.20)]
plan_ohio = plan_ohio[plan_ohio["tipo"] == "PPO"]
plan_ohio = plan_ohio[plan_ohio["nivel_metal"] == "Expanded Bronze"]
plan_ohio_perfil = plan_ohio.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_ohio_perfil = plan_ohio_perfil.loc[plan_ohio_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_ohio = pd.merge(plan_ohio_perfil, plan_ohio, how="left", on=["id","Age","IndividualRate"])
plan_ohio["id_perfil"] = 11
plan_ohio.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,28162OH0060104,60,748.72,Ohio,AultCare Bronze 7000 Select No Pediatric Dental,PPO,Expanded Bronze,AultCare Insurance Company,2023,OH,...,28162OH0060104,Rating Area 15,0,0,0,0,0,0,0,11
1,28162OH0060104,61,775.21,Ohio,AultCare Bronze 7000 Select No Pediatric Dental,PPO,Expanded Bronze,AultCare Insurance Company,2023,OH,...,28162OH0060104,Rating Area 15,0,0,0,0,0,0,0,11
2,28162OH0060104,62,792.59,Ohio,AultCare Bronze 7000 Select No Pediatric Dental,PPO,Expanded Bronze,AultCare Insurance Company,2023,OH,...,28162OH0060104,Rating Area 15,0,0,0,0,0,0,0,11


## Plan para perfil Tennessee

Perfil
- 25 to 29 years
- High school graduate (includes equivalency)	
- Soltero
- 2 o mas personas
- $75,000 to $99,999

In [312]:
plan_tennessee = tabla_general_planes[tabla_general_planes["estado_id"] == "Tennessee"]
plan_tennessee = plan_tennessee[plan_tennessee["Age"].isin(['25','26','27','28','29'])]
plan_tennessee = plan_tennessee[plan_tennessee["IndividualRate"] < (75000*0.20)]
#plan_tennessee = plan_tennessee[plan_tennessee["tipo"] == "HMO"]
plan_tennessee = plan_tennessee[plan_tennessee["nivel_metal"] == "Bronze"]
plan_tennessee_perfil = plan_tennessee.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_tennessee_perfil = plan_tennessee_perfil.loc[plan_tennessee_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_tennessee = pd.merge(plan_tennessee_perfil, plan_tennessee, how="left", on=["id","Age","IndividualRate"])
plan_tennessee["id_perfil"] = 12
plan_tennessee.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,70111TN0110032,25,254.05,Tennessee,CMS Standard Bronze,EPO,Bronze,Celtic Insurance Company,2023,TN,...,70111TN0110032,Rating Area 2,0,0,0,0,0,0,0,12
1,70111TN0110032,26,259.12,Tennessee,CMS Standard Bronze,EPO,Bronze,Celtic Insurance Company,2023,TN,...,70111TN0110032,Rating Area 2,0,0,0,0,0,0,0,12
2,70111TN0110032,27,265.19,Tennessee,CMS Standard Bronze,EPO,Bronze,Celtic Insurance Company,2023,TN,...,70111TN0110032,Rating Area 2,0,0,0,0,0,0,0,12


## Plan para perfil Texas

Perfil
- 30 to 34 years - 30 to 34 years
- High school graduate  - Bachelor's degree
- Casado - Casado
- 2 o mas personas - 2 o mas personas
- $75,000 to $99,999 - $75,000 to $99,999

In [360]:
plan_texas = tabla_general_planes[tabla_general_planes["estado_id"] == "Texas"]
plan_texas = plan_texas[plan_texas["Age"].isin(['30','31','32','33','34'])]
plan_texas = plan_texas[plan_texas["IndividualRate"] < (75000*0.16)]
plan_texas = plan_texas[plan_texas["tipo"] == "HMO"]
plan_texas = plan_texas[plan_texas["nivel_metal"] == "Silver"]
plan_texas_perfil = plan_texas.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_texas_perfil = plan_texas_perfil.loc[plan_texas_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_texas = pd.merge(plan_texas_perfil, plan_texas, how="left", on=["id","Age","IndividualRate"])
plan_texas["id_perfil"] = 13
plan_texas.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,58840TX0110060,30,352.40,Texas,Silver S: Aetna network of doctors & hospitals...,HMO,Silver,Aetna Health Inc. (a TX corp.),2023,TX,...,58840TX0110060,Rating Area 18,0,0,0,0,0,0,0,13
1,58840TX0110060,31,359.85,Texas,Silver S: Aetna network of doctors & hospitals...,HMO,Silver,Aetna Health Inc. (a TX corp.),2023,TX,...,58840TX0110060,Rating Area 18,0,0,0,0,0,0,0,13
2,58840TX0110060,32,367.30,Texas,Silver S: Aetna network of doctors & hospitals...,HMO,Silver,Aetna Health Inc. (a TX corp.),2023,TX,...,58840TX0110060,Rating Area 18,0,0,0,0,0,0,0,13


In [314]:
plan_texas1 = tabla_general_planes[tabla_general_planes["estado_id"] == "Texas"]
plan_texas1 = plan_texas1[plan_texas1["Age"].isin(['30','31','32','33','34'])]
plan_texas1 = plan_texas1[plan_texas1["IndividualRate"] < (75000*0.16)]
plan_texas1 = plan_texas1[plan_texas1["tipo"] == "EPO"]
plan_texas1 = plan_texas1[plan_texas1["nivel_metal"] == "Silver"]
plan_texas1_perfil = plan_texas1.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_texas1_perfil = plan_texas1_perfil.loc[plan_texas1_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_texas1 = pd.merge(plan_texas1_perfil, plan_texas1, how="left", on=["id","Age","IndividualRate"])
plan_texas1["id_perfil"] = 14
plan_texas1.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,29418TX0140108,30,411.93,Texas,CMS Standard Silver,EPO,Silver,Celtic Insurance Company,2023,TX,...,29418TX0140108,Rating Area 12,0,0,0,0,0,0,0,14
1,29418TX0140108,31,420.64,Texas,CMS Standard Silver,EPO,Silver,Celtic Insurance Company,2023,TX,...,29418TX0140108,Rating Area 12,0,0,0,0,0,0,0,14
2,29418TX0140108,32,429.35,Texas,CMS Standard Silver,EPO,Silver,Celtic Insurance Company,2023,TX,...,29418TX0140108,Rating Area 12,0,0,0,0,0,0,0,14


In [315]:
plan_texas_general  = pd.concat([plan_texas,plan_texas1])
#plan_texas_general  = plan_texas_general .drop(["index"], axis=1)
#plan_texas_general  = plan_texas_general .rename(columns={"id":"id_plan","level_0":"id"})
plan_texas_general.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,58840TX0110060,30,352.40,Texas,Silver S: Aetna network of doctors & hospitals...,HMO,Silver,Aetna Health Inc. (a TX corp.),2023,TX,...,58840TX0110060,Rating Area 18,0,0,0,0,0,0,0,13
1,58840TX0110060,31,359.85,Texas,Silver S: Aetna network of doctors & hospitals...,HMO,Silver,Aetna Health Inc. (a TX corp.),2023,TX,...,58840TX0110060,Rating Area 18,0,0,0,0,0,0,0,13
2,58840TX0110060,32,367.30,Texas,Silver S: Aetna network of doctors & hospitals...,HMO,Silver,Aetna Health Inc. (a TX corp.),2023,TX,...,58840TX0110060,Rating Area 18,0,0,0,0,0,0,0,13


## Plan para perfil Virginia

Perfil
- 35 to 39 years
- Bachelor's degree
- Casado
- 2 o mas personas
- $75,000 to $99,999

In [316]:
plan_virginia = tabla_general_planes[tabla_general_planes["estado_id"] == "Virginia"]
plan_virginia = plan_virginia[plan_virginia["Age"].isin(['35','36','37','38','39'])]
plan_virginia = plan_virginia[plan_virginia["IndividualRate"] < (75000*0.16)]
plan_virginia = plan_virginia[plan_virginia["tipo"] == "EPO"]
plan_virginia = plan_virginia[plan_virginia["nivel_metal"] == "Gold"]
plan_virginia_perfil = plan_virginia.groupby(["id","Age"])['IndividualRate'].min().reset_index()
plan_virginia_perfil = plan_virginia_perfil.loc[plan_virginia_perfil.groupby(["Age"])['IndividualRate'].idxmin(),["id","Age","IndividualRate"]]
plan_virginia = pd.merge(plan_virginia_perfil, plan_virginia, how="left", on=["id","Age","IndividualRate"])
plan_virginia["id_perfil"] = 15
plan_virginia.head(3)

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,38234VA0170012,35,336.80,Virginia,Gold S: Aetna network of doctors & hospitals +...,EPO,Gold,Aetna Life Insurance Company,2023,VA,...,38234VA0170012,Rating Area 7,0,0,0,0,0,0,0,15
1,38234VA0170012,36,339.01,Virginia,Gold S: Aetna network of doctors & hospitals +...,EPO,Gold,Aetna Life Insurance Company,2023,VA,...,38234VA0170012,Rating Area 7,0,0,0,0,0,0,0,15
2,38234VA0170012,37,341.21,Virginia,Gold S: Aetna network of doctors & hospitals +...,EPO,Gold,Aetna Life Insurance Company,2023,VA,...,38234VA0170012,Rating Area 7,0,0,0,0,0,0,0,15


## Planes perfil unificado

In [345]:
planes_perfiles_unificado = pd.concat([plan_arizona,
           plan_florida,
           plan_georgia_general,
           plan_illinois_general,
           plan_michigan,
           plan_carolina_general,
           plan_ohio,
           plan_tennessee,
           plan_texas_general,
           plan_virginia])
planes_perfiles_unificado

,id,Age,IndividualRate,estado_id,nombre_plan,tipo,nivel_metal,nombre_proveedor,BusinessYear,StateCode,...,PlanId,RatingAreaId,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,id_perfil
0,40702AZ0060014,25,227.59,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
1,40702AZ0060014,26,232.12,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
2,40702AZ0060014,27,237.56,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
3,40702AZ0060014,28,246.41,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
4,40702AZ0060014,29,253.66,Arizona,UHC Bronze Essential ($3 T1 Preferred Rx),HMO,Bronze,"UnitedHealthcare of Arizona, Inc.",2023,AZ,...,40702AZ0060014,Rating Area 4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,38234VA0170012,35,336.80,Virginia,Gold S: Aetna network of doctors & hospitals +...,EPO,Gold,Aetna Life Insurance Company,2023,VA,...,38234VA0170012,Rating Area 7,0,0,0,0,0,0,0,15
1,38234VA0170012,36,339.01,Virginia,Gold S: Aetna network of doctors & hospitals +...,EPO,Gold,Aetna Life Insurance Company,2023,VA,...,38234VA0170012,Rating Area 7,0,0,0,0,0,0,0,15
2,38234VA0170012,37,341.21,Virginia,Gold S: Aetna network of doctors & hospitals +...,EPO,Gold,Aetna Life Insurance Company,2023,VA,...,38234VA0170012,Rating Area 7,0,0,0,0,0,0,0,15
3,38234VA0170012,38,343.42,Virginia,Gold S: Aetna network of doctors & hospitals +...,EPO,Gold,Aetna Life Insurance Company,2023,VA,...,38234VA0170012,Rating Area 7,0,0,0,0,0,0,0,15


In [382]:
planes_perfiles_unificado.columns

Index(['id', 'Age', 'IndividualRate', 'estado_id', 'nombre_plan', 'tipo',
       'nivel_metal', 'nombre_proveedor', 'BusinessYear', 'StateCode',
       'IssuerId', 'SourceName', 'ImportDate', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'id_perfil'],
      dtype='object')

## Beneficios

In [408]:
beneficios = pd.read_csv("Benefits.csv")

In [409]:
beneficios = beneficios[beneficios["StateCode"].isin(state_filter)]
tabla_beneficios = pd.merge(planes_perfiles_unificado["id"],beneficios, how="left", left_on="id", right_on="StandardComponentId")
tabla_beneficios = tabla_beneficios[["StandardComponentId","PlanId","BenefitName","IsCovered","QuantLimitOnSvc","Explanation","IsExclFromInnMOOP","IsExclFromOonMOOP","CopayInnTier1","CopayInnTier2","CopayOutofNet","CoinsInnTier1","CoinsInnTier2","CoinsOutofNet","LimitQty","LimitUnit"]]
tabla_beneficios = tabla_beneficios[tabla_beneficios["IsCovered"] == "Covered"]
tabla_beneficios = tabla_beneficios[tabla_beneficios["IsExclFromInnMOOP"] == "No"]
# tabla_beneficios = tabla_beneficios.reset_index().reset_index()
tabla_beneficios = tabla_beneficios.drop(["PlanId","Explanation"], axis=1)
tabla_beneficios["CopayInnTier1"] = tabla_beneficios["CopayInnTier1"].apply(copay_tranform)
tabla_beneficios["CopayInnTier2"] = tabla_beneficios["CopayInnTier2"].apply(copay_tranform)
tabla_beneficios["CopayOutofNet"] = tabla_beneficios["CopayOutofNet"].apply(copay_tranform)
tabla_beneficios["CoinsInnTier1"] = tabla_beneficios["CoinsInnTier1"].fillna("0%")
tabla_beneficios["CoinsInnTier2"] = tabla_beneficios["CoinsInnTier2"].fillna("0%")
tabla_beneficios["CoinsOutofNet"] = tabla_beneficios["CoinsOutofNet"].fillna("0%")
tabla_beneficios["LimitQty"] = tabla_beneficios["LimitQty"].fillna("0")
tabla_beneficios["LimitUnit"] = tabla_beneficios["LimitUnit"].fillna("")
tabla_beneficios["IsCovered"] = tabla_beneficios["IsCovered"].fillna("Not Covered")
tabla_beneficios["QuantLimitOnSvc"] = tabla_beneficios["QuantLimitOnSvc"].fillna("No")
tabla_beneficios["IsExclFromInnMOOP"] = tabla_beneficios["IsExclFromInnMOOP"].fillna("No")
tabla_beneficios["IsExclFromOonMOOP"] = tabla_beneficios["IsExclFromOonMOOP"].fillna("No")
tabla_beneficios = tabla_beneficios.drop_duplicates()
tabla_beneficios["repetidos"] = tabla_beneficios.groupby(['StandardComponentId', 'BenefitName']).cumcount() + 1
tabla_beneficios = tabla_beneficios[tabla_beneficios["repetidos"] == 1]
tabla_beneficios = tabla_beneficios.reset_index().reset_index()
tabla_beneficios = tabla_beneficios.drop(["repetidos","index"], axis=1)
tabla_beneficios = tabla_beneficios.rename(columns={"level_0":"id"})
#tabla_beneficios[["StandardComponentId","BenefitName"]].value_counts()
tabla_beneficios = tabla_beneficios.iloc[:,0:7]
tabla_beneficios

,id,StandardComponentId,BenefitName,IsCovered,QuantLimitOnSvc,IsExclFromInnMOOP,IsExclFromOonMOOP
0,0,40702AZ0060014,Radiation,Covered,No,No,Yes
1,1,40702AZ0060014,Diabetes Education,Covered,No,No,Yes
2,2,40702AZ0060014,Prosthetic Devices,Covered,No,No,Yes
3,3,40702AZ0060014,Infusion Therapy,Covered,No,No,Yes
4,4,40702AZ0060014,Treatment for Temporomandibular Joint Disorders,Covered,No,No,Yes
...,...,...,...,...,...,...,...
833,833,38234VA0170012,Chemotherapy,Covered,No,No,Yes
834,834,38234VA0170012,Radiation,Covered,No,No,Yes
835,835,38234VA0170012,Diabetes Education,Covered,No,No,Yes
836,836,38234VA0170012,Prosthetic Devices,Covered,No,No,Yes


## Edades seguros perfiles

In [410]:
edades = planes_perfiles_unificado[["id","Age","RateEffectiveDate", "RateExpirationDate","IndividualRate"]].reset_index().reset_index()
edades = edades.rename(columns={"id":"plan_id","level_0":"id"})
edades= edades.drop(["index"], axis=1)
tabal_edades = edades[["id","plan_id","Age"]]
tabal_edades

,id,plan_id,Age
0,0,40702AZ0060014,25
1,1,40702AZ0060014,26
2,2,40702AZ0060014,27
3,3,40702AZ0060014,28
4,4,40702AZ0060014,29
...,...,...,...
72,72,38234VA0170012,35
73,73,38234VA0170012,36
74,74,38234VA0170012,37
75,75,38234VA0170012,38


## Fechas seguros perfiles

In [411]:
fechas = edades[["id","RateEffectiveDate","RateExpirationDate","IndividualRate"]].reset_index()
fechas = fechas.rename(columns={"id":"edad_seguros_id","index":"id"})
tabla_fechas = fechas[["id","RateEffectiveDate","RateExpirationDate"]]
tabla_fechas

,id,RateEffectiveDate,RateExpirationDate
0,0,2023-01-01,2023-12-31
1,1,2023-01-01,2023-12-31
2,2,2023-01-01,2023-12-31
3,3,2023-01-01,2023-12-31
4,4,2023-01-01,2023-12-31
...,...,...,...
72,72,2023-01-01,2023-12-31
73,73,2023-01-01,2023-12-31
74,74,2023-01-01,2023-12-31
75,75,2023-01-01,2023-12-31


## Tasas individuales

In [412]:
tabla_tasas = fechas[["id","IndividualRate"]].reset_index()
tabla_tasas = tabla_tasas.rename(columns={"id":"fechas_id","index":"id"})
tabla_tasas

,id,fechas_id,IndividualRate
0,0,0,227.59
1,1,1,232.12
2,2,2,237.56
3,3,3,246.41
4,4,4,253.66
...,...,...,...
72,72,72,336.80
73,73,73,339.01
74,74,74,341.21
75,75,75,343.42
